In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np 

https://github.com/sivasahukar95/CodeStore/blob/master/Scraping%20data%20from%20imdb.ipynb 

https://www.youtube.com/watch?v=I5L3OJ-xtsw

In [2]:
# Top 1000, sorted by user rating ascending 
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6422.142 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8"
}

url = "https://www.imdb.com/list/ls091520106/"

response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.text, 'html.parser')

# soup = BeautifulSoup(response.content, 'html.parser')


In [3]:
title = []
years = []
duration = []
ratings = []
metascores = []

vote_count = []
classifications = []

descriptions = []

In [4]:
# update movie title 

movie_title = soup.find_all('a', attrs={'class':'ipc-title-link-wrapper'})
for store in movie_title:
    name = store.h3.text
    title.append(name)

In [5]:
print(len(movie_title))


25


In [6]:
# update movie year, duration, classification

details = soup.findAll('div', attrs = {'class':'sc-b189961a-7 feoqjK dli-title-metadata'})

for detail in details:
    year_span = detail.span
    duration_span = year_span.find_next_sibling('span')
    class_span = duration_span.find_next_sibling('span')
    
    years.append(year_span.text)
    duration.append(duration_span.text)
    classifications.append(class_span.text)

In [7]:
# update movie rating, vote count

stars = soup.findAll('span', attrs={'class':'ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating'})

stars = [star.text for star in stars]

for star in stars:
    rating, vote = star.split('\xa0')
    ratings.append(rating)
    vote_count.append(vote.strip('()'))

In [8]:
# update descriptions 

des_lst = soup.findAll('div', attrs={'class':'ipc-html-content-inner-div'})

descriptions = [des.text for des in des_lst]


In [9]:
length = len(title)

In [10]:
# update metascore
def get_metascores():
    check = soup.find('span', attrs={'class':'sc-b189961a-1 kcfvgk'})
    ele = soup.find('span', attrs={'class':'sc-b0901df4-0 bcQdDJ metacritic-score-box'})
    for i in range(length):
        if (len(check) == 1):
            metascores.append(np.nan)
        else:
            metascores.append(ele.text)
            ele = ele.find_next('span', attrs={'class':'sc-b0901df4-0 bcQdDJ metacritic-score-box'})
        check = check.find_next('span', attrs={'class':'sc-b189961a-1 kcfvgk'}) 

In [11]:
get_metascores()

In [13]:
movie_DF = pd.DataFrame({'Name of movie': title, 'Year of relase': years, 'Watchtime': duration, 'Movie Rating': rating, 'Metascore': metascores, 'Votes': vote_count, 'Classification': classifications})

In [14]:
movie_DF.to_csv("Top_100_IMDB_Movies.csv", index=False)